In [1]:
meas = "RNA"
layer = "raw"
census = "2024-07-01"
tissue = "tongue"
var_filter = None
dask_chunk_size = 0  # Non-Dask mode, by default
n_workers = None
threads_per_worker = 1
tdb_workers = None
dashboard_port = 8787

In [2]:
# Parameters
tissue = "colon"
dask_chunk_size = 8000
n_workers = 64
threads_per_worker = 1
tdb_workers = 1
dashboard_port = 8786


In [3]:
from concurrent.futures import ThreadPoolExecutor
from os import cpu_count
from typing import Literal
import anndata as ad
import scanpy as sc
from somacore import AxisQuery
from tiledbsoma import Experiment, SOMATileDBContext

DEFAULT_CONFIG = {
    "vfs.s3.no_sign_request": "true",
    "vfs.s3.region": "us-west-2"
}
CENSUS_S3 = "s3://cellxgene-census-public-us-west-2/cell-census"

species = "homo_sapiens"
soma_uri = f"{CENSUS_S3}/{census}/soma"
exp_uri = f"{soma_uri}/census_data/{species}"

if not tdb_workers:
    tdb_workers = cpu_count()

exp = Experiment.open(
    exp_uri,
    context=SOMATileDBContext(
        tiledb_config=DEFAULT_CONFIG,
        threadpool=ThreadPoolExecutor(max_workers=tdb_workers),
    ),
)
exp

<Experiment 's3://cellxgene-census-public-us-west-2/cell-census/2024-07-01/soma/census_data/homo_sapiens' (open for 'r') (2 items)
    'ms': 's3://cellxgene-census-public-us-west-2/cell-census/2024-07-01/soma/census_data/homo_sapiens/ms' (unopened)
    'obs': 's3://cellxgene-census-public-us-west-2/cell-census/2024-07-01/soma/census_data/homo_sapiens/obs' (unopened)>

In [4]:
%%time
obs_filter = f'tissue_general == "{tissue}"' if tissue else None
query = exp.axis_query(
    measurement_name=meas,
    obs_query=AxisQuery(value_filter=obs_filter) if obs_filter else None,
    var_query=AxisQuery(value_filter=var_filter) if var_filter else None,
)

CPU times: user 13.7 ms, sys: 3.09 ms, total: 16.8 ms
Wall time: 153 ms


In [5]:
%%time
query.n_obs

CPU times: user 714 ms, sys: 1.75 s, total: 2.47 s
Wall time: 693 ms


714413

In [6]:
from dask.distributed import Client, LocalCluster
if dask_chunk_size:
    if n_workers is None:
        n_workers = cpu_count() // tdb_workers // threads_per_worker
    cluster = LocalCluster(
        n_workers=n_workers,
        threads_per_worker=threads_per_worker,
        dashboard_address=f":{dashboard_port}",
    )
    print(f"{n_workers=}, {threads_per_worker=}, {tdb_workers=}")
    client = Client(cluster)
else:
    dask_chunk_size = None
    client = None
client

n_workers=64, threads_per_worker=1, tdb_workers=1


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8786/status,
Dashboard: http://127.0.0.1:8786/status,Workers: 64
Total threads: 64,Total memory: 246.55 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:45183,Workers: 64
Dashboard: http://127.0.0.1:8786/status,Total threads: 64
Started: Just now,Total memory: 246.55 GiB
Comm: tcp://127.0.0.1:44315,Total threads: 1
Dashboard: http://127.0.0.1:38175/status,Memory: 3.85 GiB
Nanny: tcp://127.0.0.1:37909,


## HVG

In [7]:
%%time
add = query.to_anndata(layer, dask_chunk_size=dask_chunk_size)
add

CPU times: user 3.06 s, sys: 2.89 s, total: 5.95 s
Wall time: 2.28 s


AnnData object with n_obs × n_vars = 714413 × 60530
    obs: 'soma_joinid', 'dataset_id', 'assay', 'assay_ontology_term_id', 'cell_type', 'cell_type_ontology_term_id', 'development_stage', 'development_stage_ontology_term_id', 'disease', 'disease_ontology_term_id', 'donor_id', 'is_primary_data', 'observation_joinid', 'self_reported_ethnicity', 'self_reported_ethnicity_ontology_term_id', 'sex', 'sex_ontology_term_id', 'suspension_type', 'tissue', 'tissue_ontology_term_id', 'tissue_type', 'tissue_general', 'tissue_general_ontology_term_id', 'raw_sum', 'nnz', 'raw_mean_nnz', 'raw_variance_nnz', 'n_measured_vars'
    var: 'soma_joinid', 'feature_id', 'feature_name', 'feature_length', 'nnz', 'n_measured_obs'

In [8]:
%%time
sc.pp.normalize_total(add)

CPU times: user 94 ms, sys: 24.3 ms, total: 118 ms
Wall time: 112 ms


In [9]:
%%time
sc.pp.log1p(add)

CPU times: user 1.52 ms, sys: 878 μs, total: 2.4 ms
Wall time: 2.42 ms


In [10]:
%%time
hvg = sc.pp.highly_variable_genes(add, inplace=False, subset=True)
hvg

2025-01-29 21:49:03,608 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 2.69 GiB -- Worker memory limit: 3.85 GiB
2025-01-29 21:49:07,095 - distributed.worker.memory - WARNING - Worker is at 81% memory usage. Pausing worker.  Process memory: 3.15 GiB -- Worker memory limit: 3.85 GiB
2025-01-29 21:49:07,136 - distributed.worker.memory - WARNING - Worker is at 40% memory usage. Resuming worker. Process memory: 1.55 GiB -- Worker memory limit: 3.85 GiB
2025-01-29 21:49:21,636 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memo

CPU times: user 25.9 s, sys: 7.45 s, total: 33.4 s
Wall time: 44.6 s


,means,dispersions,mean_bin,dispersions_norm,highly_variable
1,0.026894,1.289713,"(-0.00486, 0.243]",0.810094,True
5,0.014958,1.619181,"(-0.00486, 0.243]",1.197983,True
6,0.131333,2.046089,"(-0.00486, 0.243]",1.700591,True
8,0.064922,1.228058,"(-0.00486, 0.243]",0.737506,True
13,0.026449,1.090968,"(-0.00486, 0.243]",0.576108,True
...,...,...,...,...,...
51229,0.018870,1.307655,"(-0.00486, 0.243]",0.831218,True
51526,0.027907,1.786350,"(-0.00486, 0.243]",1.394795,True
53899,0.022165,1.578129,"(-0.00486, 0.243]",1.149652,True
56651,0.020105,1.251600,"(-0.00486, 0.243]",0.765223,True


---
Census `tissue_general` counts, for reference:
```python
exp_obs = exp.obs.read().concat().to_pandas()
exp_obs.tissue_general.value_counts()
```
```
tissue_general
brain                       26281059
blood                       10835244
lung                         6231233
breast                       5555979
eye                          4190842
heart                        3629952
kidney                       2083054
liver                        1815408
small intestine              1237182
skin of body                 1045024
endocrine gland               979667
respiratory system            944355
bone marrow                   813373
spleen                        751041
lymph node                    717899
colon                         714413
placenta                      638786
reproductive system           570488
adrenal gland                 560114
nose                          470445
adipose tissue                468603
stomach                       446348
prostate gland                348664
fallopian tube                250178
pancreas                      250161
esophagus                     215951
digestive system              211912
musculature                   189452
large intestine               180996
pleural fluid                 179134
yolk sac                      169725
embryo                        165937
uterus                        148198
mucosa                        131978
spinal cord                   117463
exocrine gland                115722
intestine                     104490
immune system                  89046
bladder organ                  82797
vasculature                    63667
ovary                          53751
lamina propria                 45230
tongue                         45060
central nervous system         31780
esophagogastric junction       29105
axilla                         19792
pleura                         19695
skeletal system                14680
saliva                         14502
omentum                        14003
testis                         13211
tendon of semitendinosus       10533
gallbladder                     9769
scalp                           3029
ureter                          2390
Name: count, dtype: int64
```